# This implementation of the genetic algorithm for the shortfalls and is how the optimal solution was fount

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [1]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime

# Loads in the analog payloads and the shortfall scores

In [2]:
#import cell

#tech
tech_array = []

#take data from tech csv and convert it to an array
with open('Found_Technologies.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_array.append(line)


#take data from scores tech csv and convert it to an array
tech_score = []
with open('Shortfalls_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_score.append(line)

#print(tech_score)



# Prepares data from the .csv files and sets parameter values

In [3]:
#data processing cell

#Parse shortfall data 
tech_array_mod          = tech_array[1:]                                    #cut off title score
tech_mass_array         = np.asfarray([i[-1] for i in tech_array_mod])      #Masses of techs 0 indexed
tech_power_array        = np.asfarray([i[-2] for i in tech_array_mod])      #power consumption of techs 0 indexed
shortfall_array_temp    = [np.array(i[1:-2]) for i in tech_array_mod]       
shortfall_array         = []
for i in shortfall_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    shortfall_array.append(arr)

shortfall           = np.asfarray(tech_score[1:])
shortfall_matrix    = np.zeros((len(shortfall_array),len(shortfall)))
for i in range(len(shortfall_matrix)):
    for j in shortfall_array[i]:
        shortfall_matrix[i][j-1] = 1

shortfall_matrix = np.transpose(shortfall_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

shortfall_integrated    = shortfall[0:, 1]  #Scores from NASA 0 indexed
shortfall_consequence   = shortfall[0:, 2]  #Scores from consequence axis 0 indexed
shortfall_liklihood     = shortfall[0:, 3]  #Scores from liklihood axis 0 indexed

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function being optimized

In [4]:
#score function for tech

def score_tech(mission, check_mass=True):
    #weights
    w1 = 2.3  #completed_score
    w2 = 7  #integrated_score
    w3 = 4  #consequence_score
    w4 = 4  #liklihood_score

    mass_max_tech = max_mass*(1-mass_ratio) #in kg
    mass_min_tech = min_mass*(1-mass_ratio) #in kg

    #Mass check
    if check_mass and (tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) <= mass_min_tech or tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) > mass_max_tech):
        return abs(tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission))

    #normlizers
    max_number      = 93        #normalize number score 0-1 
    max_integrated  = 8.1035    #normalize integrated score 0-1
    max_consequence = 5         #normlize consquence score 0-1
    max_liklihood   = 5         #normlize consquence score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3+w4 
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W
    w4 = 10*w4/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(shortfall_matrix @ mission, a_min=None, a_max=1) #A shortfall is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of shortfalls completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_integrated_score  = shortfall_integrated  @ normalized_completed             #Generates integrated score
    completed_consequence_score = shortfall_consequence @ normalized_completed             #Generates the consquences score
    completed_liklihood_score   = shortfall_liklihood   @ normalized_completed             #Generates the liklihood score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_integrated_score/max_integrated + w3*completed_consequence_score/max_consequence + w4*completed_liklihood_score/max_liklihood
    return -score


# Initial Guess

In [6]:
#analyitcal solution
scores_per_tech = np.zeros(len(tech_array)-1)
for i in range(len(scores_per_tech)):
    input = np.zeros(len(tech_array)-1)
    input[i] = 1
    scores_per_tech[i] = score_tech(input,check_mass=False)
indcies_tech = np.argsort(scores_per_tech)
scores_per_tech_sort = np.sort(scores_per_tech) 

#rint(scores_per_tech_sort)
#print(indcies_tech)
mass = 0
bring_array_tech = np.zeros(len(tech_array)-1)
for i in range(len(indcies_tech)):
    if mass < 0.5*(max_mass+min_mass)*(1-mass_ratio):
        bring_array_tech[indcies_tech[i]] = 1
        mass = tech_mass_array.dot(bring_array_tech) + beta*tech_power_array.dot(bring_array_tech)

print(bring_array_tech)
print(score_tech(bring_array_tech))
bounds_tech = Bounds(lb=np.zeros(len(shortfall_array)), ub=np.ones(len(shortfall_array)))     #we either bring or we dont
# print(mass)
    

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
-8.051954485926398


# Trackers used to allow for multiple loops

In [ ]:
#setup
outputs_tech = []
scores_tech = []
seeds_tech = []

# Differential Evolution loop call

In [ ]:
#loop 1 min a loop
for i in range(2000):
    seed = np.random.randint(9999999)
    seeds_tech.append(seed)
    res_tech    = differential_evolution(func=score_tech, bounds=bounds_tech, x0=bring_array_tech,integrality=np.ones(len(tech_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=300,seed=seed)
    outputs_tech.append(res_tech['x'])
    scores_tech.append(res_tech['fun'])
    

# Plots occurance of analog payloads

In [ ]:
#precentage plotter
percentages = np.mean(outputs_tech, axis=0) * 100
sorted_indices = np.argsort(-percentages)
sorted_percentages = percentages[sorted_indices]
indices = [row[0] for row in tech_array_mod]
indices = [indices[i] for i in sorted_indices]
plt.figure(figsize=(15,5))
plt.bar(indices, sorted_percentages, color='grey')
plt.ylabel('Occurrence Percentage')
plt.title('Common technologies')
plt.xticks(indices, rotation=90, fontsize=7)
plt.show()

# Plots seeds with scores 

In [ ]:
#score plotter
plt.figure(figsize=(20,5))
seed_index = [str(i) for i in seeds_tech]
plt.bar(seed_index, np.asfarray(scores_tech), color='grey')
plt.xticks(seed_index, rotation=90, fontsize=5)
plt.ylabel('Scores')
plt.title('Iteration')
plt.ylim(-8.065,-8.05)
plt.yticks(np.arange(-8.065,-8.05,.0005),minor=True)
plt.grid(visible=True,which='both',axis='y')
plt.show()

# saves data to .csv fiile

In [111]:
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
#write to csv
with open("Iterations_Tech_deferred_tol01_sobol_guess" + formatted_time + '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    for i in range(len(scores_tech)):
        arr = []
        arr.append(scores_tech[i])
        arr.append(seeds_tech[i])
        for j in outputs_tech[i]:
            arr.append(j)
        wr.writerow(arr) 

# Find the best scoring seed

In [ ]:
print(len(seeds_tech))
sorted_scores = np.argsort(scores_tech)
print(scores_tech[sorted_scores[0]])
print(seeds_tech[sorted_scores[0]])